Search Engine With Tools And Agents

In [129]:
#arxiv
#tools creation 

from langchain_community.tools import ArxivQueryRun,WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper,ArxivAPIWrapper





In [130]:
api_wrapper_wiki=WikipediaAPIWrapper(top_k_results=1,doc_content_char_max=250)
wiki = WikipediaQueryRun(api_wrapper=api_wrapper_wiki)
wiki.name



'wikipedia'

In [131]:
api_wrapper_arxiv=ArxivAPIWrapper(top_k_results=1,doc_content_chars_max=250)
arxiv=ArxivQueryRun(api_wrapper=api_wrapper_arxiv)
arxiv.name

'arxiv'

In [132]:
tools=[wiki,arxiv]

In [133]:
## custom tools[rag tools]
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter


In [134]:
from dotenv import load_dotenv
import os
load_dotenv()

os.environ['HF_TOKEN']=os.getenv("HF_TOKEN")
loader = WebBaseLoader("https://docs.smith.langchain.com/")
docs = loader.load()
documents= RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200).split_documents(docs)
from langchain_huggingface.embeddings.huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2") 
vectordb = FAISS.from_documents(documents, embeddings) 
retriever = vectordb.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000028AE8F372F0>, search_kwargs={})

In [135]:
from langchain.tools.retriever import create_retriever_tool


In [136]:
retriever_tool = create_retriever_tool(
    retriever=retriever,
    name="LangSmith_Search",
    description="Search any information about LangSmith"
)


In [137]:
tools = [wiki,arxiv,retriever_tool]

In [138]:
# run all this tools with agents and llms
#tools , llm -->agentexecutor
from dotenv import load_dotenv
from langchain_groq import ChatGroq
load_dotenv()
import os


In [139]:
groq_api_key=os.getenv("QROQ_API_KEY")
###print("Groq API Key:", groq_api_key)


In [140]:
llm =ChatGroq(groq_api_key=groq_api_key,model_name="llama-3.1-8b-instant")

In [141]:
#prompt template
from langchain import hub
prompt = hub.pull("hwchase17/openai-functions-agent")


In [142]:
prompt.messages


[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [143]:
#agents

from langchain.agents import create_openai_tools_agent
agent = create_openai_tools_agent(llm,tools,prompt)
agent

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages

In [144]:
## agent executer 
from langchain.agents import AgentExecutor
agent_executor=AgentExecutor(agent =agent ,tools=tools,verbose=True)




In [145]:
print(agent_executor.input_keys)


[]


In [146]:
response = agent_executor.invoke({
    "page_content": "Content from your PDF or document",
    "input": "Tell me about LangSmith"
})

print(response)




> Entering new AgentExecutor chain...

Invoking: `LangSmith_Search` with `{'query': 'LangSmith'}`


LangSmith docs - Docs by LangChainSkip to main contentDocs by LangChain home pageLangSmithSearch...⌘KGitHubTry LangSmithTry LangSmithSearch...NavigationLangSmith docsGet startedObservabilityEvaluationPrompt engineeringDeploymentAgent BuilderPlatform setupOverviewPlansCreate an account and API keyAccount administrationOverviewSet up a workspaceManage organizations using the APIManage billingSet up resource tagsUser managementReferenceLangSmith Python SDKLangSmith JS/TS SDKLangGraph Python SDKLangGraph JS/TS SDKLangSmith APIAPI reference for LangSmith DeploymentAdditional resourcesReleases & changelogsData managementAccess control & AuthenticationScalability & resilienceFAQsRegions FAQPricing FAQLangSmith docsCopy pageCopy pageLangSmith provides tools for developing, debugging, and deploying LLM applications.
It helps you trace requests, evaluate outputs, test prompts, and manage deploym

In [147]:
response = agent_executor.invoke({
    "input": "Tell me about the paper 1706.03762"
})

print(response)



> Entering new AgentExecutor chain...

Invoking: `arxiv` with `{'query': '1706.03762'}`


Published: 2023-08-02
Title: Attention Is All You Need
Authors: Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Lukasz Kaiser, Illia Polosukhin
Summary: The dominant sequence transduction models are based on cIt seems like the title, authors and summary are not available for the paper 1706.03762. The output suggests that it has something to do with sequence transduction.

> Finished chain.
{'input': 'Tell me about the paper 1706.03762', 'output': 'It seems like the title, authors and summary are not available for the paper 1706.03762. The output suggests that it has something to do with sequence transduction.'}
